In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import cupy as cp
import mpmath as mp
import concurrent.futures

In [3]:
# Skip this cell if CSR2D/ is already added under PYHTONPATH 
import os, sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)  

In [4]:
from csr2d.core import psi_s, psi_x
from csr2d.dist import lambda_p_Gauss

In [5]:
from mpmath import ellipf, ellipe

# Simple example

In [6]:
def haha(a,b):
    return a*b

In [7]:
N = 500
A = np.arange(N)
B = 2*np.arange(N)

In [8]:
%time
haha(A,B);

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


In [9]:
%time
DD = np.arange(0.) # empty array 

with concurrent.futures.ProcessPoolExecutor() as executor:
    for AA, BB, CC in zip(A, B, executor.map(haha, A, B)):
        #print('haha(%s, %s) is %d ' % (AA, BB, CC))
        DD = np.append(DD,CC)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.81 µs


In [10]:
#DD

# Testing for psi_x

In [20]:
gamma = 500
beta = (1-1/gamma**2)**(1/2)
rho = 1
sigmax = 10E-6
sigmaz = 10E-6

# Adjust the step size here
# Small steps might greatly increase computation time...

N = 100
dz = 1.0E-4/N
dx = 1.0E-4/N

In [21]:
zvec2 = np.arange(-10*sigmaz, 10*sigmaz, dz)
xvec2 = np.arange(-10*sigmax, 10*sigmax, dx)
zm2, xm2 = np.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')

In [23]:
%%time
## Direct numpy vectorization 
result = psi_x(zm2,xm2,beta);

CPU times: user 1min 42s, sys: 25.3 ms, total: 1min 42s
Wall time: 1min 42s


In [22]:
%%time
# With Multi-processing
beta_grid = beta*np.ones(zm2.shape)
DD = np.arange(0.)
with concurrent.futures.ProcessPoolExecutor() as executor:
    for PSI_X in executor.map(psi_x, zm2, xm2, beta_grid):
        DD = np.append(DD,PSI_X)

CPU times: user 166 ms, sys: 329 ms, total: 495 ms
Wall time: 1min 30s


In [15]:
DD.reshape(zm2.shape)

array([[ 2.00014178,  2.0001382 ,  2.00013444, ...,  1.99987892,
         1.99987473,  1.99987071],
       [ 2.00014072,  2.00013721,  2.00013358, ...,  1.99987938,
         1.99987532,  1.99987134],
       [ 2.00013959,  2.00013616,  2.0001326 , ...,  1.99987989,
         1.99987588,  1.99987203],
       ...,
       [-1.96801008, -1.96905972, -1.97010781, ..., -2.02538592,
        -2.02634854, -2.02730969],
       [-1.96879593, -1.96981927, -1.97084114, ..., -2.02478908,
        -2.02572951, -2.02666853],
       [-1.96953558, -1.9705342 , -1.97153141, ..., -2.02422621,
        -2.02514569, -2.02606382]])

In [26]:
%%time
with concurrent.futures.ProcessPoolExecutor() as executor:
    kaka = np.array( list(executor.map(psi_x, zm2, xm2, beta_grid)))

CPU times: user 146 ms, sys: 353 ms, total: 499 ms
Wall time: 1min 32s
